In [ ]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, ConversationKGMemory
from langchain.chains import ConversationChain

In [2]:
load_dotenv()
api_key = os.getenv('GROQ_API_KEY')

In [3]:
chat = ChatGroq(temperature=0, groq_api_key=api_key, model_name="mixtral-8x7b-32768")

In [4]:
prompt_template = ChatPromptTemplate.from_template(
    template="""
You are a highly intelligent and efficient personal assistant. Your tasks include:
- Helping the user manage their daily tasks and schedule.
- Providing useful recommendations based on previous interactions.
- Remembering important details from past conversations.
- Responding in a friendly, professional, and concise manner.

Consider the user's context from previous conversations stored in memory.

{history}

User: {input}
Assistant:"""
)


# ConversationBufferMemory

In [5]:
# Initialize memory
memory = ConversationBufferMemory(return_messages=True)

C:\Users\smkaa\AppData\Local\Temp\ipykernel_56444\1740195726.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [6]:
# Save custom input-output pairs to memory
memory.save_context({"input": "What's your name?"}, {"output": "My name is Karthik and I am a graduate student at the University of Illinois at Urbana-Champaign."})
memory.save_context({"input": "What can you do?"}, {"output": "I am very good at data engineering and machine learning."})
memory.save_context({"input": "Tell me a joke."}, {"output": "Girish is gay"})
memory.save_context({"input": "What is your favorite language?"}, {"output": "Tamil"})

In [7]:
# Use the saved memory in a conversation chain
conversation_bm = ConversationChain(
    llm=chat,  # your LLM object
    verbose=True,
    memory=memory,
    prompt=prompt_template
)

# Example of interacting with the chain using the stored memory
response = conversation_bm.run(input="What's your name?")
print(response)

response = conversation_bm.run(input="Tell me a joke.")
print(response)

# Memory will recall the context from previous inputs
response = conversation_bm.run(input="Do you remember what I asked earlier?")
print(response)


C:\Users\smkaa\AppData\Local\Temp\ipykernel_56444\1977001814.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation_bm = ConversationChain(
C:\Users\smkaa\AppData\Local\Temp\ipykernel_56444\1977001814.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = conversation_bm.run(input="What's your name?")




> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
You are a highly intelligent and efficient personal assistant. Your tasks include:
- Helping the user manage their daily tasks and schedule.
- Providing useful recommendations based on previous interactions.
- Remembering important details from past conversations.
- Responding in a friendly, professional, and concise manner.

Consider the user's context from previous conversations stored in memory.

[HumanMessage(content="What's your name?", additional_kwargs={}, response_metadata={}), AIMessage(content='My name is Karthik and I am a graduate student at the University of Illinois at Urbana-Champaign.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What can you do?', additional_kwargs={}, response_metadata={}), AIMessage(content='I am very good at data engineering and machine learning.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke.', additional_kwargs={

# ConversationWindowBufferMemory

In [12]:
# Initialize memory
memory_window = ConversationBufferWindowMemory(k=2,return_messages=True)

In [13]:
# Save custom input-output pairs to memory
memory_window.save_context({"input": "Remind me to buy groceries."}, {"output": "Noted. I will remind you to buy groceries."})
memory_window.save_context({"input": "Schedule a meeting with John tomorrow."}, {"output": "Meeting scheduled with John at 10 AM tomorrow."})
memory_window.save_context({"input": "What does john do "}, {"output": "He works as a software engineer at Google."})
memory_window.save_context({"input": "What was the first thing I asked you?"}, {"output": "I'm sorry, I can only remember recent conversations. You asked about the meeting with John."})

In [14]:
# Load the memory variables
print(memory_window.load_memory_variables({}))

{'history': [HumanMessage(content='What does john do ', additional_kwargs={}, response_metadata={}), AIMessage(content='He works as a software engineer at Google.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What was the first thing I asked you?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm sorry, I can only remember recent conversations. You asked about the meeting with John.", additional_kwargs={}, response_metadata={})]}


In [ ]:
# Use the saved memory in a conversation chain
conversation_bwm = ConversationChain(
    llm=chat,  # your LLM object
    verbose=True,
    memory=memory_window,
    prompt=prompt_template
)

# Example of interacting with the chain using the stored memory
response = conversation_bwm.run(input="When do i have the meeting with john")
print(response)

response = conversation_bwm.run(input="What does john do")
print(response)

# Memory will recall the context from previous inputs
response = conversation_bwm.run(input="Do you remember what I asked earlier?")
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
You are a highly intelligent and efficient personal assistant. Your tasks include:
- Helping the user manage their daily tasks and schedule.
- Providing useful recommendations based on previous interactions.
- Remembering important details from past conversations.
- Responding in a friendly, professional, and concise manner.

Consider the user's context from previous conversations stored in memory.

[HumanMessage(content='What does john do ', additional_kwargs={}, response_metadata={}), AIMessage(content='He works as a software engineer at Google.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What was the first thing I asked you?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm sorry, I can only remember recent conversations. You asked about the meeting with John.", additional_kwargs={}, response_metadata={})]

User: When do i have the meeting with john
Assistant:

>

## ConversationSummaryMemory

In [17]:
memory_summary = ConversationSummaryMemory(llm=chat)

C:\Users\smkaa\AppData\Local\Temp\ipykernel_56444\3139242821.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory_summary = ConversationSummaryMemory(llm=chat)


In [18]:
# Saving custom fitness tracking interactions
memory_summary.save_context({"input": "I ran 5 kilometers today."}, {"output": "Great job! I've recorded your 5 km run."})
memory_summary.save_context({"input": "I did yoga for 30 minutes."}, {"output": "Awesome! Yoga is great for flexibility and relaxation. I've noted it down."})
memory_summary.save_context({"input": "What have I done for fitness this week?"}, {"output": "This week, you ran 5 kilometers and did yoga for 30 minutes."})

In [19]:
memory_summary.load_memory_variables({})

{'history': 'The human shared with the AI their recent accomplishment of running 5 kilometers and doing yoga for 30 minutes, to which the AI responded positively, highlighted the benefits of these activities, and recorded them. When the human inquired about their fitness activities for the week, the AI provided a summary, mentioning the run and yoga session.'}

In [20]:
# Use the saved memory in a conversation chain
conversation_sm = ConversationChain(
    llm=chat,  # your LLM object
    verbose=True,
    memory=memory_summary,
    prompt=prompt_template
)

# Example of interacting with the chain using the stored memory
response = conversation_sm.run(input="How many kilometers did I run this week?")
print(response)

response = conversation_sm.run(input="Did I do any yoga this week?")
print(response)

response = conversation_sm.run(input="What can I do to improve my fitness?")
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
You are a highly intelligent and efficient personal assistant. Your tasks include:
- Helping the user manage their daily tasks and schedule.
- Providing useful recommendations based on previous interactions.
- Remembering important details from past conversations.
- Responding in a friendly, professional, and concise manner.

Consider the user's context from previous conversations stored in memory.

The human shared with the AI their recent accomplishment of running 5 kilometers and doing yoga for 30 minutes, to which the AI responded positively, highlighted the benefits of these activities, and recorded them. When the human inquired about their fitness activities for the week, the AI provided a summary, mentioning the run and yoga session.

User: How many kilometers did I run this week?
Assistant:

> Finished chain.
You've run 5 kilometers this week, specifically on the day we previously discussed. Remember, 